In [1]:
from Transformations import *
from torch import Tensor
import torch as th

# Manipulator Differential Kinematics
### Part I: Kinematics, Velocity, and Applications

Manipulator kinematics is concerned with the motion of each link within a manipulator without considering mass or force. This work is an introduction to modelling manipulator kinematics using the __elementary transform sequence (ETS)__.

A serial-link manipulator, which we refer to as a manipulator, is the formal name for a robot that comprises a chain of rigid links and joints, it may contain branches, but it can not have closed loops. Each joint provides one degree of freedom, which may be a __prismatic joint__ providing translational freedom or a __revolute joint__ providing rotational freedom. __The base frame__ of a manipulator represents the reference frame of the first link in the chain, while the last link is known as __the end-effector__.

The __elementary transform sequence (ETS)__ provides a universal method for describing the kinematics of any manipulator. This intuitive and systematic approach can be applied with a simple wal-through procedure. The resulting sequence comprises a number of elementary transforms - translations and rotations - from the base frame to the robot's end-effector.

The __zero-angle configuration__ of a manipulator is the state of the robot when all its joint angles and translational distances are set to zero. This is set by the manufacturers of the manipulator.

We use the notations where $\{a\}$ denotes a coordinate frame, and $^{a}\mathbf{T}_{b}$ is a relative pose of rigid-body transformation of $\{b\}$ with respect to $\{a\}$.

### Forward Kinematics

The forward kinematics of a manipulator provides a non-linear mapping
$$
    ^{0}\mathbf{T}_{e}\left(t\right) = \mathcal{K}\left(\mathbf{q}\left(t\right)\right)
$$
between the joint space and cartesian task space, where $\mathbf{q}\left(t\right)=\left[ q_1\left(t\right), q_2\left(t\right), \hdots, q_n\left(t\right) \right] \in \mathbb{R}^{n}$ is the vecto of joint generalised cordinates, $n$ is the number of joints, and $^{0}\mathbf{T}_{e} \in \mathbf{SE}(3)$ is a homogenous transformation matrix representing the pose of the robot's end-effector in the world-coordinate frame. The ETS model defines $\mathcal{K}\left・\right)$ as the product of $M$ elementary transforms $\mathbf{E}_i \in \mathbf{SE}(3)$
$$\begin{*align}
    ^{0}\mathbf{T}_{e}\left( t \right) &= \mathbf{E}_1\left( \eta_1 \right) \mathbf{E}_2\left( \eta_2 \right) \hdots \mathbf{E}_M\left( \eta_M \right) \\
    &= \prod^{M}_{i=1} \mathbf{E}_i\left( \eta_i \right)
\end{*align}$$
where each of the elementary transforms $\mathbf{E}_i$ can be a pure translation along, or a pure rotation about the local x-, y- or z-axis by an amount $\eta_i$. Explicitly, each transform is one of the following:
$$
    \mathbf{E}_i = \{\; \mathbf{T}_{\mathbf{t}_x}\left(\eta_i\right),\; \mathbf{T}_{\mathbf{t}_y}\left(\eta_i\right),\; \mathbf{T}_{\mathbf{t}_y}\left(\eta_i\right),\; \mathbf{T}_{\mathbf{R}_x}\left(\eta_i\right),\; \mathbf{T}_{\mathbf{R}_y}\left(\eta_i\right),\; \mathbf{T}_{\mathbf{R}_y}\left(\eta_i\right)\; \}
$$
where each of the matrices' argument $\eta_i$ is either a constant $c_i$ (translational offset or rotation) or a joint variable $q_j{t}$:
$$
    \eta_i = \{\; c_i, q_j\left(t\right)\; \}
$$
and the joint variable is
$$
    q_j\left(t\right) = \begin{cases}
        \theta_j(t) &\hspace{5mm} \text{for a revolute joint} \\
        d_j(t) &\hspace{5mm} \text{for a prismatic joint}
    \end{cases}
$$

An __ETS__ description does not require intermediate link frames, but it does not preclude their introduction. The relative transform between link frames $a$ and $b$ is simply a subset of the __ETS__
$$
    ^{a}\mathbf{T}_{b} = \prod^{\mu(b)}_{\mu(a)} \mathbf{E}_i\left(\eta_i\right)
$$
where the function $\mu{j}$ returns the index of the term in the ETS expression.

### Deriving the Manipulator Jacobian

##### First Derivative of a Pose
Consider the end-effector pose $\mathbf{T}$, its derivative with respect to time is
$$
    \dot{\mathbf{T}} = \frac{\text{d}\mathbf{T}}{\text{d}t} = \frac{\partial \mathbf{T}}{\partial q_1}\dot{q_1} + \hdots + \frac{\partial \mathbf{T}}{\partial q_n}\dot{q_n} \in \mathbb{R}^{4 \times 4}
$$
where each $\frac{\partial \mathbf{T}}{\partial q_i} \in \mathbf{R}^{4 \times 4}$. The information in $\mathbf{T}$ is non-minimal, and redundant, as is the information in $\mathbf{T}'$.
$$
    \mathbf{T} = \begin{bmatrix}
        \mathbf{R} & \mathbf{t} \\
        \mathbf{0} & \mathbf{1}
    \end{bmatrix}, \hspace{10mm}  \dot{\mathbf{T}} = \begin{bmatrix}
        \dot{\mathbf{R}} & \dot{\mathbf{t}} \\
        \mathbf{0} & \mathbf{0}
    \end{bmatrix}
$$
where $\mathbf{R} \in \mathbf{SO}(3)$, $\dot{\mathbf{R}} \in \mathbb{R}^{3 \times 3}$, and $\mathbf{t}, \dot{\mathbf{t}} \in \mathbf{R}^3$. We can denote the partial derivative in partitioned form as:
$$
    \frac{\partial \mathbf{T}}{\partial q_j} = \begin{bmatrix}
        \mathbf{J}_{R_j} & \mathbf{J}_{t_j} \\
        \mathbf{0} & \mathbf{0}
    \end{bmatrix}
$$
where $\mathbf{J}_{R_j} \in \mathbb{R}^{3 \times 3}$ and $\mathbf{J}_{t_j} \in \mathbb{R}^{3 \times 1}$ and then we write the transformation derivative as:
$$
    \dot{\mathbf{T}} = \begin{bmatrix}
        \dot{\mathbf{R}} & \dot{\mathbf{t}} \\
        \mathbf{0} & \mathbf{0}
    \end{bmatrix} = \begin{bmatrix}
        \mathbf{J}_{R_1} & \mathbf{J}_{t_1} \\
        \mathbf{0} & \mathbf{0}
    \end{bmatrix} \dot{q}_1 + \hdots + \begin{bmatrix}
        \mathbf{J}_{R_n} & \mathbf{J}_{t_n} \\
        \mathbf{0} & \mathbf{0}
    \end{bmatrix} \dot{q}_n
$$
and write a matrix equation for each non-zero partition
$$
    \dot{\mathbf{R}} = \mathbf{J}_{R_1}\dot{q}_1 + \hdots + \mathbf{J}_{R_n}\dot{q}_n \\
    \dot{\mathbf{t}} = \mathbf{J}_{t_1}\dot{q}_1 + \hdots + \mathbf{J}_{t_n}\dot{q}_n
$$
where each term represents the condition to end-effector velocity due to motion of the corresponding joint.

Where $\mathbf{J}_{v}(\mathbf{q}) \in \mathbb{R}^{3 \times n}$ is the __translational part of the manipulator Jacobian matrix__, we can write
$$\begin{align}
    \dot{\mathbf{t}} &= \left[ \mathbf{J}_{t_1} \hdots \mathbf{J}_{t_n} \right] \begin{bmatrix}
        \dot{q}_1 \\
        \vdots \\
        \dot{q}_n
    \end{bmatrix} \\
    &= \mathbf{J}_{v}(\mathbf{q})\dot{\mathbf{q}}
\end{align}$$

Using the identity $\dot{\mathbf{R}} = [\mathbf{\omega}]_{\times}\mathbf{R}$ where $\mathbf{\omega} \in \mathbb{R}^3$ is __the angular velocity__, and $[\mathbf{\omega}]_{\times} \in \mathbf{so(3)}$ is a skew-symmetric matrix. We can then write
$$
    [\mathbf{\omega}]_{\times} \mathbf{R} = \mathbf{J}_{R_1}\dot{q}_1 + \hdots + \mathbf{J}_{R_n}\dot{q}_n
$$
and rearrange to
$$
    [\mathbf{\omega}]_{\times} = (\mathbf{J}_{R_1}\mathbf{R}^{\top})\dot{q}_1 + \hdots + (\mathbf{J}_{R_n}\mathbf{R}^{\top})\dot{q}_n \in \mathbf{so(3)}.
$$
This $3 \times 3$ matrix equation has only 3 unique equations, so applying the inverse skew oerator to both sides we have
$$\begin{align}
    \mathbf{\omega} &= \vee_{\times}(\mathbf{J}_{R_1}\mathbf{R}^{\top})\dot{q}_1 + \hdots + \vee_{\times}(\mathbf{J}_{R_n}\mathbf{R}^{\top})\dot{q}_n \\
    &= \left[ \vee_{\times}(\mathbf{J}_{R_1}\mathbf{R}^{\top})\; \hdots\;  \vee_{\times}(\mathbf{J}_{R_n}\mathbf{R}^{\top}) \right] \begin{bmatrix}
        \dot{q}_1 \\
        \vdots \\
        \dot{q}_n
    \end{bmatrix} \\
    &= \mathbf{J}_{\omega}(\mathbf{q})\dot{\mathbf{q}}
\end{align}$$
where $\mathbf{J}_{\omega}(\mathbf{q}) \in \mathbb{R}^{3 \times n}$ is __the rotational part of the manipulator Jacobian__.

We can combine the translational and rotational parts of the manipulator Jacobian and write:
$$
    ^{0}\mathbf{\mathcal{v}} = \begin{bmatrix}
        \mathbf{v} \\
        \mathbf{\omega}
    \end{bmatrix} = \begin{bmatrix}
        \mathbf{J}_{\vartheta}(\mathbf{q}) \\
        \mathbf{J}_{\omega}(\mathbf{q})
    \end{bmatrix} \dot{\mathbf{q}}
$$
which expresses end-effector spatial velocity in the world frame in terms of joint velocity and
$$
    ^{0}\mathbf{J}(\mathbf{q}) = \begin{bmatrix}
        \mathbf{J}_{\vartheta}(\mathbf{q}) \\
        \mathbf{J}_{\omega}(\mathbf{q})
    \end{bmatrix} \in \mathbb{R}^{6 \times n}
$$
is the manipulator Jacobian matrix expressed in the world-coordinate frame. In a more compact form, we have:
$$
    ^{0}\mathbf{v} = ^{0}\mathbf{J}(\mathbf{q})\dot{\mathbf{q}}
$$

##### First Derivative of an Elementary Transform
Before differentiating the ETS to find the manipulator Jacobian, it is useful to consider the derivative of a single Elementary Transform.

__Derivative of a pure Rotation__: The derivative of a rotation matrix with respect to the rotation angle $\theta$ is required when considering a revolute joint and can be shown to be:
$$
    \frac{\text{d}\mathbf{R}(\theta)}{\text{d}\theta} = \left[ \hat{\mathbf{\omega}} \right]_{\times}\mathbf{R}(\theta(t))
$$
where the unit vector $\hat{\mathbf{\omega}}$ is the joint rotation axis. Re-araanging, we have:
$$
    \hat{\mathbf{\omega}} = \vee_{\times} \left( \frac{\text{d}\mathbf{R}(\theta)}{\text{d}\theta} \mathbf{R}(\theta(t))^{\top} \right)
$$
since $\mathbf{R} \in \mathbf{SO}(3)$, then $\mathbf{R}^{-1}=\mathbf{R}^{\top}$. Derivative of each elementary rotation with respect to a rotation angle is
$$
    \frac{\text{d}\mathbf{T}_{\mathbf{R}_x}(\theta)}{\text{d}\theta} = \begin{bmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & -1 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
    \end{bmatrix} \mathbf{T}_{\mathbf{R_x}} = [\hat{\mathbf{R}}_x] \mathbf{T}_{\mathbf{R_x}}, \\
    \frac{\text{d}\mathbf{T}_{\mathbf{R}_y}(\theta)}{\text{d}\theta} = \begin{bmatrix}
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 \\
        -1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
    \end{bmatrix} \mathbf{T}_{\mathbf{R_y}} = [\hat{\mathbf{R}}_y] \mathbf{T}_{\mathbf{R_y}}, \\
    \frac{\text{d}\mathbf{T}_{\mathbf{R}_z}(\theta)}{\text{d}\theta} = \begin{bmatrix}
        0 & -1 & 0 & 0 \\
        1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
    \end{bmatrix} \mathbf{T}_{\mathbf{R_z}} = [\hat{\mathbf{R}}_z] \mathbf{T}_{\mathbf{R_z}}
$$
where each of the augmented skew-symmetric matrices $[\hat{\mathbf{R}}]$ above corresponds to one of the generators of $\mathbf{SE}(3)$ which lies in $\mathbf{se}(3)$, the tangent space of $\mathbf{SE}(3)$.

##### Derivative of a Pure Translation
Derivatives of each elementary translation with respect to a translation below:
$$
    \frac{\text{d}\mathbf{T}_{\mathbf{t}_x}(d)}{\text{d}d} = \begin{bmatrix}
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
    \end{bmatrix} = [\hat{\mathbf{t}}_x], \\
    \frac{\text{d}\mathbf{T}_{\mathbf{t}_y}(d)}{\text{d}d} = \begin{bmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
    \end{bmatrix} = [\hat{\mathbf{t}}_y], \\
    \frac{\text{d}\mathbf{T}_{\mathbf{t}_z}(d)}{\text{d}d} = \begin{bmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 \\
    \end{bmatrix} = [\hat{\mathbf{t}}_z]
$$
where each of the augmented skew symmetric matrices $[\hat{\mathbf{t}}]$ above are the remaining three generators of $\mathbf{SE}(3)$ which lie in $\mathbf{se}(3)$.

##### The Manipulator Jacobian
To find out how the $j^{\text{th}}$ joint affects the end-effector pose, we apply the chain rule to the ETS:
$$\begin{align}
    \frac{\partial\mathbf{T}(\mathbf{q})}{\partial q_j} &= \frac{\partial}{\partial q_j} \left( \mathbf{E}_1(\eta_1)\mathbf{E}_2(\eta_2)\hdots\mathbf{E}_M(\eta_M)\right) \\
    &= \prod^{\mu(j)-1}_{i=1} \mathbf{E}_i(\eta_i) \frac{\text{d}\mathbf{E}_{\mu(j)}(q_j)}{\text{d}q_j} \prod^{M}_{i=\mu(j)+1} \mathbf{E}_i(\eta_i)
\end{align}$$

Using the previously defined $\mathbf{J}_{\omega}$ and $\mathbf{J}_{v}$, we can form the angular velocity components of the $j^{\text{th}}$ column of the manipulator Jacobian
$$
    \mathbf{J}_{\omega_j}(\mathbf{q})=\vee_{\times} \left( \rho \left(\frac{\partial \mathbf{T}(\mathbf{q})}{\partial q_j}\right) \rho \left( \mathbf{T}(\mathbf{q})^{\top} \right) \right)
$$
and the translational velocity compinent of the $j^{\text{th}}$ column of the manipulator Jacobian is
$$
    \mathbf{J}_{v_j}(\mathbf{q}) = \tau \left( \frac{\partial \mathbf{T}(\mathbf{q})}{\partial q_j} \right)
$$
Stacking the translation and angular velocity components, the $j^{\text{th}}$ column of the manipulator Jacobian becomes
$$
    \mathbf{J}_j(\mathbf{q}) = \begin{bmatrix}
        \mathbf{J}_{v_j}(\mathbf{q}) \\
        \mathbf{J}_{\omega_j}(\mathbf{q})
    \end{bmatrix} \in \mathbb{R}^{6}
$$
where the full manipulator Jacobian is
$$
    \mathbf{J}(\mathbf{q}) = \left[\; \mathbf{J}_1(\mathbf{q})\; \hdots\; \mathbf{J}_n(\mathbf{q})\; \right] \in \mathbb{R}^{6 \times n}
$$

##### Fast Manipulator Jacobian
Calculating the manipulator Jacobian using the above method is easy to understand, but has $\mathcal{O}(n^2)$ time complexity.

### Manipulator jacobian Applications

##### Numerical Inverse Kinematics

Inverse kinematics deals in the problem of determining the corresponding joint coordinates, given some end-effector pose. There are two approaches to solving the inverse kinematics: analytical and numerical.

Analytical formulas must be pre-generated for a given manipulator and in some cases may not exist. Analytical solutions generally cannot optimise for additionally criteria such as joint limits.

Numerical inverse kinematics use an iterative technique and can additionally consider extra constraints such as collision avoidance, joint limit avoidance, or manipulability.

Using the __Newton-Raphson__ (NR) method for inverse kinematics to find the joint coordinates which correspond to some end-effector pose $^{0}\mathbf{T}_{e^*}$, the NR method seeks to minimise an error function
$$
    E = \frac{1}{2}\mathbf{e}^{\top}\mathbf{W}_e\mathbf{e}
$$
where
$$
    \mathbf{e} = \begin{bmatrix}
        \tau\left(^{0}\mathbf{T}_{e^*}\right) - \tau\left(^{0}\mathbf{T}_{e}\right) \\
        \alpha \left( \rho\left(^{0}\mathbf{T}_{e^*}\right)\rho\left(^{0}\mathbf{T}_{e}\right)^{\top} \right)
    \end{bmatrix} \in \mathbb{R}^6
$$
where $\alpha(*) : \mathbf{SO}(3) \mapsto \mathbb{R}^3$ transforms a rotation matrix to its Euler vector equivalent and $\mathbf{W}_e = \text{diag}(\mathbf{w}_e)$ with $\mathbf{w}_e \in \mathbb{R}^n \ge 0$ is a diagonal weighting matrix which prioritises the corresponding error term. To achieve this, we iterate upon the following
$$
    \mathbf{q}_{k+1} = \mathbf{q}_k + ^{0}\mathbf{J}(\mathbf{q})^{-1}\mathbf{e}_k
$$

We define $\rho(\mathbf{T})=\mathbf{R}=\{r_{ij}\}$ and
$$
    \mathcal{l}=\begin{bmatrix}
        r_{32}-r_{23} \\
        r_{13}-r_{31} \\
        r_{21}-r_{12}
    \end{bmatrix}
$$
If $\mathbf{R}$ is not a diagonal matrix then the angle-axis equivalent of $\mathbf{R}$ is calculated as
$$
    \alpha(\mathbf{R})=\frac{\text{atan2}\left(\lVert \mathcal{l} \rVert, r_{11}+r_{22}+r_{33}-1\right)} {\lVert \mathcal{l} \rVert} \mathcal{l}
$$
If $\mathbf{R}$ is a diagonal matrix then we use different formulas. For the case where $(r_{11}, r_{22}, r_{33}) = (1, 1, 1)$ then $\alpha(\mathbf{R})=(0,0,0)^{\top}$ otherwise
$$
    \alpha(\mathbf{R})=\frac{\pi}{2} \begin{bmatrix}
        r_{11}+1 \\
        r_{22}+1 \\
        r_{33}+1
    \end{bmatrix}
$$

When using the NR method, the initial joint coordinates $q_0$, should correspond to a non-singular manipulator pose, since it uses the manipulator Jacobian. When the problem is solvable, it converges very quickly. However, this method frequently fails to converge on the goal. We can improve the solvability of the problem by using the Gauss-Newton (GN) method
$$
    \mathbf{q}_{k+1} = \mathbf{q}_k + \left(\mathbf{J}(\mathbf{q}_k)^{\top}\mathbf{W}_e\mathbf{J}(\mathbf{q}_k)\right)^{-1}\mathbf{q}_k \\
    \mathbf{g}_k = \mathbf{J}(\mathbf{q}_k)^{\top}\mathbf{W}_e\mathbf{e}_k
$$
where $\mathbf{J}=^{0}\mathbf{J}$ is the base-frame manipulator Jacobian. This can also be affected by $\mathbf{J}(\mathbf{q_k})$ being singular making GN solution infeasible.

We can further improve the solvability through the Levenberg-Marquardt (LM) method
$$
    \mathbf{q}_{k+1} = \mathbf{q}_k + \left(\mathbf{A}_k\right)^{-1}\mathbf{q}_k \\
    \mathbf{A}_k = \mathbf{J}(\mathbf{q}_k)^{\top}\mathbf{W}_e\mathbf{J}(\mathbf{q}_k) + \mathbf{W}_n
$$
where $\mathbf{W}_n = \text{diag}(\mathbf{w}_n)(\mathbf{w}_n \in \mathbb{R}^{n}_{> 0})$ is a diagonal damping matrix. The damping matrix ensures that $\mathbf{A}_k$ is non-singular and positive definite. The performance of the LM method largely depends on the choice of $\mathbf{W}_n$.

Wampler proposed $\mathbf{w}_n$ to be a constant, Chan and Lawrence proposed a damped least-squares method with
$$
    \mathbf{W}_n = \lambda E_k \mathbf{1}_n
$$
where $\lambda$ is a constant which does not have much influence on performance. Sugihara proposed
$$
    \mathbf{W}_n = E_k \mathbf{1}_n + \text{diag}(\tilde{\mathbf{w}}_n)
$$
where $\tilde{\mathbf{w}}_n \in \mathbb{R}^{n}$, $\hat{w}_{n_i} = l^2 \sim 0.001l^2$, and $l$ is the length of a typical link within the manipulator.

An important point to note is that the above methods are subject to local minima and in some cases will fail to converge on the solution. The choice of the initial joint configuration $\mathbf{q}_0$ is important. An alternative approach is to re-start an IK problem with a new random $\mathbf{q}_0$ after a few $20 \sim 50$ iterations rather than persist with a single attempt method $500 \sim 5000$ iterations. This is a simple but effective method of performing a global search for the IK solution.

##### Manipulator Performance Metrics

Manipulator performance metrics seek to quantify the performance of a manipulator in a given configuration. Two common manipulator performance metrics based on the manipulator Jacobian. Firstly, the metrics are unitless, and the upper bound of a metric depends on the manipulator kinematic model (i.e. joint types and link lengths). Consequently, metrics computed for different manipulators are not directly comparable. Secondly, the manipulator Jacobian contains three rows corresponding to translational rates, and three rows corresponding to angular rates. Therefore, any metrics using the whole Jacobian will produce a non-homogenous result due to the mixed units. Depending on the manipulator scale, this can cause either the translational or rotational component to dominate result. In general, the most intuitive use of performance metrics comes from using only the translational or rotational rows of the manipulator Jacobian (where the choice of which depends on the use case), and only using the metric on a manipulator comprising a single joint type.

__Manipulability Index__: The Yoshikawa manipulability index is the most widely used and accepted performance metric. The index is calculated as
$$
    m(\mathbf{q}) = \sqrt{\text{det}\left( \hat{\mathbf{J}}(\mathbf{q})\hat{\mathbf{J}}(\mathbf{q})^{\top} \right)}
$$
where $\hat{\mathbf{J}}(\mathbf{q}) \in \mathbb{R}^{3 \times n}$ is either the translational or rotational rows of $\mathbf{J}(\mathbf{q})$ causing $m(\mathbf{q})$ to describe the corresponding component of manipulability. The scalar $m(\mathbf{q})$ describes the volume of a 3-dimensional ellipsoid - if this ellipsoid is close to spherical, then the manipulator can achieve any arbitrary end-effector (translational or rotational depending on $\hat{\mathbf{J}}(\mathbf{q})$) velocity.

The ellipsoid is described by three radii aligned with its principal axes.A small radii indicates the robot's inability to achieve a velocity in the corresponding direction. At a singularity, the ellipsoid's radius becomes zero along the corresponding axis and the volume becomes zero. If the manipulator is well conditioned, these ellipsoids will have a larger volume. Therefore, the manipulability index is essentially a measure of how easily a manipulator can achieve an arbitrary velocity.